In [4]:
from arcgis.features import FeatureLayer
mhm_url = "https://services4.arcgis.com/feSRYIeirl5nUF92/arcgis/rest/services/MHM/FeatureServer/0"

layer = FeatureLayer(mhm_url)


In [5]:
df = layer.query(where = "protocol = 'mosquito_habitat_mapper'").sdf
df

""


In [7]:
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(threshold = (0, 250))
def get_groups(threshold = 10):
    global df
    # groups / filters suspected events
    suspect_df = df.groupby(by=['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude']).filter(lambda x: len(x) > threshold)

    # identify groups
    suspect_groups = suspect_df.groupby(by = ['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude'])

    # create a groups dataset
    groups_data = pd.DataFrame()
    for group, df in suspect_groups:
        new_df = df.copy(True)
        new_df["Group Name"] = str(group)
        groups_data = groups_data.append(new_df, ignore_index = True)
    return groups_data


interactive(children=(IntSlider(value=10, description='threshold', max=250), Output()), _dom_classes=('widget-…

In [8]:
from arcgis.gis import GIS

gis = GIS( url="https://igestrategies.maps.arcgis.com", username="Piphi5")

Enter password:  ·········


In [9]:
from IPython.display import display, clear_output
import ipywidgets as widgets

from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry
from arcgis.gis import GIS



# Create & Display Map
viz_map = gis.map('MHM Groups')
viz_map.add_layer(layer)
display(viz_map)

MapView(layout=Layout(height='400px', width='100%'))